![banner.png](banner.png)

<h2 style="color:#de4a48; background-color:#fce19a; padding: 10px; text-align:left; border: 1px solid #fce19a;">Imports</h2>

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, split, min, max, count
import os

In [2]:
spark = SparkSession.builder \
    .appName("IMDbProject") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

In [ ]:
# spark.conf.set("spark.sql.shuffle.partitions", "4")
# spark.conf.set("spark.executor.memory", "2g")

<h2 style="color:#de4a48; background-color:#fce19a; padding: 10px; text-align:left; border: 1px solid #fce19a;">Load Datasets</h2>

In [5]:
# Paths
IMDB_PATH = '../data/raw/imdb'
TMDB_PATH = '../data/raw/tmdb'
RAW_PARQUET_PATH = '../data/raw_parquet'
PROCESSED_PATH = '../data/processed'

In [6]:
# Make sure output folders exist
os.makedirs(RAW_PARQUET_PATH, exist_ok=True)
os.makedirs(PROCESSED_PATH, exist_ok=True)

imdb_files = {
    "basics": "title.basics.tsv",
    "akas": "title.akas.tsv",
    "crew": "title.crew.tsv",
    "episode": "title.episode.tsv",
    "principals": "title.principals.tsv",
    "ratings": "title.ratings.tsv",
    "names": "name.basics.tsv"
}

### Load IMDB data to Spark Dataframes

In [7]:
imdb_dfs = {
    key: spark.read.csv(
        os.path.join(IMDB_PATH, filename),
        sep='\t',
        header=True,
        inferSchema=True,
        nullValue='\\N'
    )
    for key, filename in imdb_files.items()
}

for name, df in imdb_dfs.items():
    print(f"IMDb - {name}:")
    df.show(5)

IMDb - basics:
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|   NULL|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|   NULL|             5|     Animation,Short|
|tt0000003|    short|        Poor Pierrot|      Pauvre Pierrot|      0|     1892|   NULL|             5|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|   NULL|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|   NULL|             1

### Save unfiltered IMDB files to Parquet

In [9]:
spark.version

'3.5.5'

In [8]:
for name, df in imdb_dfs.items():
    parquet_path = os.path.join(RAW_PARQUET_PATH, f'{name}.parquet')
    df.write.mode('overwrite').parquet(parquet_path)
    print(f"Saved {name} to {parquet_path}")

Py4JJavaError: An error occurred while calling o84.parquet.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.ja

### Load TMDB data to Spark Dataframes

In [ ]:
tmdb_path = os.path.join(TMDB_PATH, 'TMDB_movie_dataset_v11.csv')

tmdb_df = spark.read.csv(
    tmdb_path,
    header=True,
    inferSchema=True,
    nullValue='\\N'
)

print("TMDb Data Preview:")
tmdb_df.show(5)

### Save TMDB file to Parquet

In [ ]:
TMDB_PARQUET_PATH = '../data/raw_parquet'
os.makedirs(TMDB_PARQUET_PATH, exist_ok=True)

tmdb_df.write.mode('overwrite').parquet(os.path.join(TMDB_PARQUET_PATH, 'TMDB_movie_dataset_v11.parquet'))
print("Saved TMDb to parquet")

IMDb - basics:
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|   NULL|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|   NULL|             5|     Animation,Short|
|tt0000003|    short|        Poor Pierrot|      Pauvre Pierrot|      0|     1892|   NULL|             5|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|   NULL|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|   NULL|             1

<h2 style="color:#de4a48; background-color:#fce19a; padding: 10px; text-align:left; border: 1px solid #fce19a;">Exploratory Data Analysis</h2>

### Loading Using Parquet Files

### Preliminary EDA

In [ ]:
imdb_basics = imdb_dfs['basics']

# Total counts for each dataset
print("Row counts per file:")
for name, df in imdb_dfs.items():
    print(f"{name}: {df.count():,} rows")

# Date range in imdb_basics
date_range = imdb_basics.select(min(col("startYear")).alias("min_year"), max(col("startYear")).alias("max_year"))
date_range.show()

# Top genres
genres_exploded = imdb_basics.withColumn("genre", explode(split(col("genres"), ",")))
top_genres = genres_exploded.groupBy("genre").count().orderBy(col("count").desc())
top_genres.show(10)

Available IMDb DataFrames:
- basics
- akas
- crew
- episode
- principals
- ratings
- names


### Filter to `movies` only

#### Record Counts

In [ ]:
print(f"IMDb Movies Count: {imdb_basics.count()}")
print(f"TMDb Movies Count: {tmdb_movies.count()}")


#### Schema Checks

In [10]:
print("IMDb Basics Schema:")
imdb_basics.printSchema()

print("TMDb Schema:")
tmdb_movies.printSchema()


IMDb Basics Schema:


NameError: name 'imdb_basics' is not defined

### 1. Highest Rated Movies in 2023

**Balancing Ratings and Number of Votes:**



### 2. Most Popular Actors/Actresses in 2023

### 3. User-Movie Trends

### 4. Metric of a 'Hit Movie'

<h2 style="color:#de4a48; background-color:#fce19a; padding: 10px; text-align:left; border: 1px solid #fce19a;">Predictive Modelling</h2>